# Report 03
### Yuzhong Huang

## The cat problem from ThinkBayes chap05

Suppose you are having a dinner party with 10 guests and 4 of them are allergic to cats. Because you have cats, you expect 50% of the allergic guests to sneeze during dinner. At the same time, you expect 10% of the non-allergic guests to sneeze. What is the distribution of the total number of guests who sneeze?

In [1]:
# Solution

## Lincoln index problem from ThinkBayes chap09

Note that the question is originally from http://www.johndcook.com/blog/2010/07/13/lincoln-index/

"Suppose you have a tester who finds 20 bugs in your program. You want to estimate how many bugs are really in the program. You know there are at least 20 bugs, and if you have supreme confidence in your tester, you may suppose there are around 20 bugs. But maybe your tester isn't very good. Maybe there are hundreds of bugs. How can you have any idea how many bugs there are? There’s no way to know with one tester. But if you have two testers, you can get a good idea, even if you don’t know how skilled the testers are.
Suppose two testers independently search for bugs. Let k1 be the number of errors the first tester finds and k2 the number of errors the second tester finds. Let c be the number of errors both testers find. The Lincoln Index estimates the total number of errors as k1 k2 / c [I changed his notation to be consistent with mine]."
So if the first tester finds 20 bugs, the second finds 15, and they find 3 in common, we estimate that there are about 100 bugs. What is the Bayesian estimate of the number of errors based on this data?

In [1]:
# Solution

## Original problems

There is a frog and a bug in a room. The room is 20 feet by 20 feet. Suppose the coordinate system’s origin is on the lower left corner of the room. And (20, 20) marks the upper right corner of the room. The frog was initially sitting on point (3, 10), and heard some noise of the bug. When the frog moved to (7, 13), the noise appeared to be louder. Suppose the bug is not moving. Where could that bug be hiding? 

In [2]:
# Solution

## Project progress

I’m thinking about a problem when waiting for the Wellesley shuttle near Chapel under the big oak tree. I saw oaks falling from the tree. Imagine that the tree covers a circle of certain radius. Every minute there is an acorn fall down from the tree. And the chance of an acorn will fall at the covered circle can be modeled as a two dimensional normal distribution. What is the chance that I will be hit by an acorn given the position I’m standing, the radius of me and how long I’m standing. I think it will be an interesting question for me to solve for the final project, so that students waiting for wellesley shuttle has a better idea the chance they will get hit by acorns. 